<a href="https://colab.research.google.com/github/SueZZD/Grover-s-algorithm/blob/main/GroverAlgo_SuedaZeynepDemirta%C5%9F_SalihaNur%C3%96zdemir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install pylatexenc
from qiskit import *
# import from the new visualization module
from qiskit.visualization import plot_bloch_multivector

import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=37bb1b396363e5a69f9903335f1e8eca141d88b8fbc874ecf61020d464cb2ae5
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.8 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
# Importing Aer from qiskit_aer
from qiskit_aer import Aer
# Instead of importing from qiskit.primitives, import from qiskit
from qiskit import QuantumCircuit, transpile
#Import the mct function:
from qiskit.circuit.library import MCXGate
# Şifre: 0110 (decimal 6)
secret_string = '0110'

def construct_oracle(secret_string):
    # 4 ana qubit + 1 yardımcı qubit (faz için)
    oracle = QuantumCircuit(5)

    # Şifreyi 0 olan bitlerde X kapısı ile çevir
    for i, bit in enumerate(secret_string):
        if bit == '0':
            oracle.x(i)

    # Faz uygulamak için kontrollü Z (mct - multi-controlled Toffoli)
    oracle.h(4)
    # Use MCXGate instead of mct:
    mct_gate = MCXGate(4) # Create a MCXGate with 4 control qubits
    oracle.append(mct_gate, [0, 1, 2, 3, 4]) # Append the gate to the circuit
    oracle.h(4)

    # X kapılarını geri al (uncompute)
    for i, bit in enumerate(secret_string):
        if bit == '0':
            oracle.x(i) # This line was likely missing, causing the error

    oracle.name = "Oracle"
    return oracle


In [ ]:
from qiskit_aer import Aer

def diffuser(nqubits):
    diffuser = QuantumCircuit(nqubits)
    for qubit in range(nqubits - 1):
        diffuser.h(qubit)
        diffuser.x(qubit)
    diffuser.h(nqubits - 2)
    mct_gate = MCXGate(nqubits - 1)  # n-1 kontrol qubitli Toffoli
    diffuser.append(mct_gate, list(range(nqubits)))
    diffuser.h(nqubits - 2)
    for qubit in range(nqubits - 1):
        diffuser.x(qubit)
        diffuser.h(qubit)
    return diffuser

# Grover devresini oluştur
n = 5  # 4 ana qubit + 1 yardımcı
grover = QuantumCircuit(n, 4)

# Süperpozisyon ve yardımcı qubit ayarı
grover.h(range(4))
grover.x(4)
grover.h(4)

# Oracle ve diffuser ekle (2 tur)
oracle = construct_oracle(secret_string)
# Instead of appending the oracle as a single gate, decompose it:
grover.compose(oracle, range(n), inplace=True)
grover.compose(diffuser(n), range(n), inplace=True)
grover.compose(oracle, range(n), inplace=True)
grover.compose(diffuser(n), range(n), inplace=True)

# Ölçüm yap
grover.measure(range(4), range(4))

# Simülasyon
backend = Aer.get_backend('qasm_simulator')
# Transpile the circuit to the basis gates supported by the backend
grover_transpiled = transpile(grover, backend)
result = backend.run(grover_transpiled, shots=1024).result()
counts = result.get_counts()

# Sonuçları yazdır ve görselleştir
print("Ölçüm Sonuçları:", counts)
plot_histogram(counts)
plt.show()

Ölçüm Sonuçları: {'1010': 70, '1110': 61, '1111': 65, '0001': 63, '0101': 85, '1001': 66, '0110': 70, '1000': 62, '1100': 58, '1101': 68, '0000': 66, '0010': 60, '1011': 55, '0011': 50, '0111': 56, '0100': 69}
